In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from geopy.geocoders import Nominatim

# Load the dataset into a DataFrame
data = pd.read_csv('covidnewpractice.csv')

# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Create a geocoder instance
geolocator = Nominatim(user_agent='heatmap_app', timeout=10)

# Create a figure
fig = go.Figure()

# Iterate over unique recorded dates
for date in data['date'].dt.date.unique():
    # Filter data for the specific date
    filtered_data = data[data['date'].dt.date == date]

    # Convert country names to latitude and longitude coordinates
    coordinates = filtered_data['location'].apply(lambda x: geolocator.geocode(x).point if geolocator.geocode(x) else None)
    filtered_data.loc['latitude'] = [coord[0] if coord else None for coord in coordinates]
    filtered_data.loc['longitude'] = [coord[1] if coord else None for coord in coordinates]

    # Drop rows with missing coordinates
    filtered_data = filtered_data.dropna(subset=['latitude', 'longitude'])

    # Add a heatmap trace for the date
    fig.add_trace(go.Densitymapbox(
        lat=filtered_data['latitude'],
        lon=filtered_data['longitude'],
        z=filtered_data['new_cases_smoothed'],  # Your value column for color intensity
        radius=10,
        coloraxis="coloraxis"
    ))

# Create a choropleth map trace
fig.add_trace(px.choropleth_mapbox(
    data_frame=data,
    geojson='your_geojson_file.json',  # Path to the GeoJSON file
    locations='location',  # Column in the DataFrame containing the country names
    featureidkey='properties.name',  # Key in the GeoJSON file for country names
    color='new_cases_smoothed',  # Column in the DataFrame for color intensity
    color_continuous_scale='Viridis',
    mapbox_style="open-street-map",
    zoom=0,
    opacity=0.5
).data[0])

# Set up the layout
fig.update_layout(
    mapbox=dict(
        center=dict(lat=0, lon=0),
        zoom=0
    ),
    coloraxis=dict(colorscale='Viridis', cmin=0, cmax=data['new_cases_smoothed'].max()),
)

# Create a slider to control the date
slider = dict(
    active=0,
    steps=[go.layout.slider.Step(method='restyle', args=[{'visible': [True if i == j else False for i in range(len(fig.data))]}],
                                 label=str(date.date()), value=str(date.date()))
           for j, date in enumerate(data['date'].dt.date.unique())]
)

# Update the layout with the slider
fig.update_layout(sliders=[slider])

# Export the figure as an HTML file
pio.write_html(fig, 'recent_covid_heatmap.html')


ValueError: cannot set a row with mismatched columns